## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-29-01-16-50 +0000,nypost,Distressing NTSB simulation shows pilots’ view...,https://nypost.com/2026/01/28/us-news/devastat...
1,2026-01-29-01-09-27 +0000,cbc,What we know about plans to open the Rafah bor...,https://www.cbc.ca/news/world/rafah-crossing-w...
2,2026-01-29-01-06-15 +0000,nyt,Rubio Says Venezuela Will Submit Monthly Budge...,https://www.nytimes.com/2026/01/28/us/politics...
3,2026-01-29-01-03-16 +0000,nypost,Billboard touting ‘Trump Accounts’ airs in Tim...,https://nypost.com/2026/01/28/us-news/billboar...
4,2026-01-29-01-00-00 +0000,wsj,Minneapolis Police and Residents Forge New Com...,https://www.wsj.com/us-news/minneapolis-police...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2488/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,49
39,minneapolis,24
47,ice,20
43,new,17
66,fed,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
79,2026-01-28-22-56-41 +0000,nypost,"Alex Pretti appears to spit at ICE, kick out t...",https://nypost.com/2026/01/28/us-news/alex-pre...,129
388,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...,126
97,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...,122
180,2026-01-28-18-58-42 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,119
366,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
79,129,2026-01-28-22-56-41 +0000,nypost,"Alex Pretti appears to spit at ICE, kick out t...",https://nypost.com/2026/01/28/us-news/alex-pre...
388,126,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...
366,67,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
208,59,2026-01-28-17-30-00 +0000,wsj,The Bank of Canada kept its policy rate unchan...,https://www.wsj.com/articles/bank-of-canada-st...
310,55,2026-01-28-11-10-37 +0000,nyt,Iran Killed Thousands of Protesters. Here Are ...,https://www.nytimes.com/2026/01/28/world/middl...
92,48,2026-01-28-22-35-00 +0000,wsj,The Federal Reserve entered a new holding patt...,https://www.wsj.com/economy/central-banking/fe...
360,47,2026-01-28-06-26-29 +0000,nypost,South Korea’s former first lady Kim Keon Hee s...,https://nypost.com/2026/01/28/world-news/south...
298,44,2026-01-28-12-01-00 +0000,wsj,"If American politics were a card game (hey, it...",https://www.wsj.com/politics/policy/the-two-bi...
97,43,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...
71,42,2026-01-28-23-13-22 +0000,startribune,Minneapolis police say FBI has taken over lead...,https://www.startribune.com/minneapolis-police...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
